In [2]:
# pip install sentence-transformers

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

/home/gwei4/miniconda3/envs/kaggle_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-29 17:39:34,708] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/gwei4/miniconda3/envs/kaggle_env/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/gwei4/minicon

In [4]:
data_dir      = Path("data")
parquet_path  = data_dir / "lol_champions_data.parquet"
out_sbert_emb = data_dir / "sbert_embeddings.npy"
out_ids       = data_dir / "champion_ids.npy"

In [5]:
df = pd.read_parquet(parquet_path)
df["text_to_embed"] = (
    df["name"].fillna("")     + " — " +
    df["role"].fillna("")     + "\n" +
    df["race"].fillna("")     + "\n" +
    df["short_bio"].fillna("")+ "\n" +
    df["full_story"].fillna("")
)
texts = df["text_to_embed"].tolist()

In [7]:
# Load SBERT (“all-MiniLM-L6-v2” is 384-dim, very fast)
sbert = SentenceTransformer("models/sbert_all-MiniLM-L6-v2")

In [8]:
sbert_embeddings = sbert.encode(texts, show_progress_bar=True, batch_size=16)

Batches: 100%|██████████| 11/11 [00:00<00:00, 17.89it/s]


In [9]:
np.save(out_sbert_emb, sbert_embeddings)
if not out_ids.exists():
    np.save(out_ids, df.index.values)

print(f"Saved SBERT embeddings → {out_sbert_emb} (shape: {sbert_embeddings.shape})")

Saved SBERT embeddings → data/sbert_embeddings.npy (shape: (170, 384))
